In [2]:
!pip -q install -U uproot awkward

import os, re, glob, shutil
import uproot


In [3]:
DATA_DIR = "/kaggle/input/datasets/katakuricharlotte/doublemuon2016g"
OUT_DIR  = "/kaggle/working/root_converted"
os.makedirs(OUT_DIR, exist_ok=True)

all_files = sorted([p for p in glob.glob(DATA_DIR + "/*") if os.path.isfile(p)])
print("Found:", len(all_files))
for p in all_files:
    print(p)


Found: 5
/kaggle/input/datasets/katakuricharlotte/doublemuon2016g/CMS_Run2016G_DoubleMuon_NANOAOD_UL2016_MiniAODv2_NanoAODv9-v2_2430000_file_index.json_0
/kaggle/input/datasets/katakuricharlotte/doublemuon2016g/CMS_Run2016G_DoubleMuon_NANOAOD_UL2016_MiniAODv2_NanoAODv9-v2_2430000_file_index.json_1
/kaggle/input/datasets/katakuricharlotte/doublemuon2016g/CMS_Run2016G_DoubleMuon_NANOAOD_UL2016_MiniAODv2_NanoAODv9-v2_2430000_file_index.json_10
/kaggle/input/datasets/katakuricharlotte/doublemuon2016g/CMS_Run2016G_DoubleMuon_NANOAOD_UL2016_MiniAODv2_NanoAODv9-v2_2430000_file_index.json_11
/kaggle/input/datasets/katakuricharlotte/doublemuon2016g/CMS_Run2016G_DoubleMuon_NANOAOD_UL2016_MiniAODv2_NanoAODv9-v2_2430000_file_index.json_12


In [ ]:
def is_root_file(path):
    with open(path, "rb") as f:
        return f.read(4) == b"root"   # ROOT magic bytes [web:45]

converted, skipped = [], []

for src in all_files:
    if not is_root_file(src):
        skipped.append(src)
        continue

    name = os.path.basename(src)

    # Fix the wrong ".json_<N>" marker in the filename
    name = name.replace(".json_", "_")   # "...file_index_0"
    name = name.replace(".json", "")     # safety

    if not name.lower().endswith(".root"):
        name = name + ".root"

    dst = os.path.join(OUT_DIR, name)
    shutil.copyfile(src, dst)
    converted.append(dst)

print("Converted:", len(converted))
print("Skipped:", len(skipped))
print("Outputs:")
for p in converted:
    print(p)


In [ ]:
!ls -lah /kaggle/working/root_converted

for p in converted:
    with uproot.open(p) as f:
        print("\nFILE:", os.path.basename(p))
        print("keys:", f.keys(recursive=False, cycle=False)[:20])
        if "Events" in f:
            print("Events entries:", f["Events"].num_entries)
